In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('finalcluster').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 20:34:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/29 20:34:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/29 20:34:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
data1 = spark.read.csv('hack_data.csv', inferSchema=True, header=True)

In [9]:
from pyspark.ml.feature import (VectorAssembler, StringIndexer)

In [10]:
data1.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [11]:
loca_model = StringIndexer(inputCol='Location', outputCol='LocationIndex')
local_str = loca_model.fit(data1)

In [12]:
data1 = local_str.transform(data1)

In [27]:
data1.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'LocationIndex']

In [29]:
assembler = VectorAssembler(inputCols= ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed',
 'LocationIndex'], outputCol='features')

In [30]:
final_data = assembler.transform(data1)

In [32]:
final_data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, LocationIndex=88.0, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37, 88.0]))]

In [34]:
from pyspark.ml.feature import StandardScaler

In [35]:
Sta_obj = StandardScaler(inputCol='features', outputCol='standardfetures')

In [36]:
Sts_model = Sta_obj.fit(final_data)

In [44]:
final_data = Sts_model.transform(final_data)

IllegalArgumentException: requirement failed: Output column standardfetures already exists.

In [46]:
final_data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, LocationIndex=88.0, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37, 88.0]), standardfetures=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963, 1.7258]))]

In [47]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- LocationIndex: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- standardfetures: vector (nullable = true)



In [48]:
from pyspark.ml.clustering import KMeans

In [51]:
kmeans2 = KMeans(featuresCol='standardfetures', k=2)
kmeans3 = KMeans(featuresCol='standardfetures', k=3)

In [52]:
kmodle2 = kmeans2.fit(final_data)
kmodle3 = kmeans3.fit(final_data)

23/12/29 22:13:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/29 22:13:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [54]:
kmodle2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [55]:
kmodle3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+

